In [13]:
import asyncio
import requests
from datetime import datetime, timedelta
import pandas as pd
from scipy.integrate import trapezoid
import math
from statistics import mean
from io import StringIO
import logging

In [14]:
logging.basicConfig(format='%(levelname)-8s [%(filename)s:%(lineno)d] %(message)s',level=logging.DEBUG)

In [15]:
async def send_get_request(url:str='http://participant0.local:5000/',endpoint:str='',type:str='json',key=None,timeout=1):
        """Send GET request to the IP."""
        # get own data
        max_tries = 3

        if key:
            headers = {"Authorization": f"Bearer {key}"}
        else:
            headers = {}
            
        for attempt in range(max_tries):
            try:
                response = requests.get(f"{url}{endpoint}",headers=headers, timeout=timeout)
                response.raise_for_status()
                if type == 'json':
                    res= parse_datetimes(convert_bools(response.json()))
                elif type == 'text':
                    res= response.text
                else:
                    res= response.status_code
                break
            except requests.exceptions.HTTPError as e:
                logging.error(f"HTTP error occurred: {e}")
            except Exception as e:
                logging.error(f'{e}')
                if attempt == max_tries-1: # try up to 3 times
                    return None
                else:
                    logging.debug('SLEEEEEEEEEEEEEEEEEPING')
                    await asyncio.sleep(1+attempt)
        return res

# # convert datetimes to iso formatted strings
# def convert_datetimes(obj):
#     if isinstance(obj, dict):
#         return {k: convert_datetimes(v) for k, v in obj.items()}
#     elif isinstance(obj, list):
#         return [convert_datetimes(i) for i in obj]
#     elif isinstance(obj, datetime):
#         return obj.isoformat()
#     else:
#         return obj

# convert to datetimes from iso formatted strings
def parse_datetimes(obj):
    if isinstance(obj, dict):
        return {k: parse_datetimes(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [parse_datetimes(i) for i in obj]
    elif isinstance(obj, str):
        try:
            return datetime.fromisoformat(obj)
        except ValueError:
            return obj
    else:
        return obj

 # Function to recursively convert "true"/"false" strings to Booleans
def convert_bools(obj):
    if isinstance(obj, dict):
        return {k: convert_bools(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_bools(elem) for elem in obj]
    elif obj == "true":
        return True
    elif obj == "false":
        return False
    else:
        return obj

In [38]:
# get performance
async def getAvgReduction(eType,mon):
    #get performance data
    performance = await send_get_request('http://participant0.local:5000/api/performance',type='json')

    #filter performance by event type
    filteredPerformance = {}
    for k,v in performance.items():
        if v['event']==eType:
            #filter by month
            if int(k.split('-')[1]) == mon:
                filteredPerformance[k] = v

    # get all flexAvgs in kW
    allAvgFlexKW = []
    for k,v in filteredPerformance.items():
        allAvgFlexKW.append(v['flexW_avg']*.001)

    if len(allAvgFlexKW)>0:
        totAvgFlexKW = mean(allAvgFlexKW)
    else:
        totAvgFlexKW = 0
    return totAvgFlexKW
    
perfDict = await getAvgReduction('dlrp',datetime.now().month)

DEBUG    [connectionpool.py:241] Starting new HTTP connection (1): participant0.local:5000
DEBUG    [connectionpool.py:544] http://participant0.local:5000 "GET /api/performance HTTP/1.1" 200 1008


In [39]:
mon = datetime.now().month
c = await getAvgReduction('csrp',mon)
d = await getAvgReduction('dlrp',mon)



DEBUG    [connectionpool.py:241] Starting new HTTP connection (1): participant0.local:5000
DEBUG    [connectionpool.py:544] http://participant0.local:5000 "GET /api/performance HTTP/1.1" 200 1008
DEBUG    [connectionpool.py:241] Starting new HTTP connection (1): participant0.local:5000
DEBUG    [connectionpool.py:544] http://participant0.local:5000 "GET /api/performance HTTP/1.1" 200 1008


TypeError: tuple expected at most 1 argument, got 2

In [43]:
(c * 18,d * 18)

(0, 2.2608295853645837)